In [4]:
import os
import numpy as np
import IPython
import copy
from shutil import copyfile

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base

import sys
sys.dont_write_bytecode=True

PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from influence.inceptionModel import BinaryInceptionModel
from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
import influence.experiments
from influence.dataset import DataSet
# from influence.dataset_poisoning import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.iter_attack import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.Progress import *

from load_animals import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
img_side = 299
num_channels = 3
 
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]

weight_decay = 0.001

num_classes = 2
max_lbfgs_iter = 1000

num_train_ex = 400 #900
num_test_ex = 200 #300
batch_size = 100

dataset_name = 'dogfish_%s_%s' % (num_train_ex, num_test_ex)
# data_sets = load_animals(
#     num_train_ex_per_class=num_train_ex_per_class, 
#     num_test_ex_per_class=num_test_ex_per_class,
#     classes=['dog', 'fish'])
# data_sets = load_dogfish_with_orig_and_koda()
# X, Y = load_koda()
# print(X.shape, Y.shape)
data_sets = new_load_dogfish_with_koda(num_train_ex, num_test_ex)

full_graph = tf.Graph()
top_graph = tf.Graph()

# dummy_data_sets = load_dummy(num_train_ex_per_class=num_train_ex_per_class, 
#     num_test_ex_per_class=num_test_ex_per_class,
#     classes=['dog', 'fish'])

Loading Koda from disk...


In [ ]:
print('*** Full:')
with full_graph.as_default():
    full_model_name = '%s_inception_wd-%s' % (dataset_name, weight_decay)
    full_model = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)
    
    for data_set, label in [
        (data_sets.train, 'train'),
        (data_sets.test, 'test')]:

        inception_features_path = 'output/%s_inception_features_new_%s.npz' % (dataset_name, label)
        if not os.path.exists(inception_features_path):

            print('Inception features do not exist. Generating %s...' % label)
            data_set.reset_batch()
            
            num_examples = data_set.num_examples
            assert num_examples % batch_size == 0

            inception_features_val = generate_inception_features(
                full_model, 
                data_set.x, 
                data_set.labels, 
                batch_size=batch_size)
            
            np.savez(
                inception_features_path, 
                inception_features_val=inception_features_val,
                labels=data_set.labels)
            
train_f = np.load('output/%s_inception_features_new_train.npz' % dataset_name)
inception_X_train = DataSet(train_f['inception_features_val'], train_f['labels'])
test_f = np.load('output/%s_inception_features_new_test.npz' % dataset_name)
inception_X_test = DataSet(test_f['inception_features_val'], test_f['labels'])

validation = None

inception_data_sets = base.Datasets(train=inception_X_train, validation=validation, test=inception_X_test)

print('*** Top:')
with top_graph.as_default():
    top_model_name = '%s_inception_onlytop_wd-%s' % (dataset_name, weight_decay)
    input_dim = 2048
    top_model = BinaryLogisticRegressionWithLBFGS(
        input_dim=input_dim,
        weight_decay=weight_decay,
        max_lbfgs_iter=max_lbfgs_iter,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=inception_data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=False,
        train_dir='output',
        log_dir='log',
        model_name=top_model_name)
    weights = top_model.retrain_and_get_weights(inception_X_train.x, inception_X_train.labels)
    orig_weight_path = 'output/inception_weights_%s.npy' % top_model_name
    np.save(orig_weight_path, weights)
    
with full_graph.as_default():
    full_model.load_weights_from_disk(orig_weight_path, do_save=False, do_check=True)

### Create poisoned dataset
print('Creating poisoned dataset...')

step_size = 0.02

num_train = len(data_sets.train.labels)
num_test = len(data_sets.test.labels)
max_num_to_poison = 10

### Try attacking each test example individually

orig_X_train = np.copy(data_sets.train.x)
orig_Y_train = np.copy(data_sets.train.labels)

test_indices_to_attack = [3]

for test_idx in test_indices_to_attack:

    print('****** Attacking test_idx %s ******' % test_idx)
    test_description = test_idx

    # If this has already been successfully attacked, skip
    filenames = [filename for filename in os.listdir('./output') if (
        (('%s_attack_testidx-%s_trainidx-' % (full_model.model_name, test_description)) in filename) and        
        (filename.endswith('stepsize-%s_proj_final.npz' % step_size)))]
        # and (('stepsize-%s_proj_final.npz' % step_size) in filename))] # Check all step sizes        
    if len(filenames) > 0:
        print('test_idx %s has already been successfully attacked. Skipping...')
        continue
        
    # Use top model to quickly generate inverse HVP
    test_inception_single_data = DataSet(np.array([inception_X_test.x[test_idx, :]]), np.array([inception_X_test.labels[test_idx]]))
    test_single_data = DataSet(np.array([data_sets.test.x[test_idx, :]]), np.array([data_sets.test.labels[test_idx]]))
    with top_graph.as_default():
        get_hvp(
            top_model,
            test_inception_single_data, inception_X_train,
            test_description=test_description,
            force_refresh=True)
    copyfile(
        'output/%s-test-%s.npz' % (top_model_name, test_description),
        'output/%s-test-%s.npz' % (full_model_name, test_description))
        
    # Use full model to select indices to poison
    with full_graph.as_default():
        influence_on_test_loss_val = get_influence_on_test_loss(
                    full_model,
                    test_single_data, 
                    np.arange(num_train), data_sets.train,
                    test_description,
                    force_refresh = False)
        # save into file for caching ##### TODO
        print("finished calculating grad_wrt_input_val")
        all_indices_to_poison = select_examples_to_attack(
            full_model, 
            max_num_to_poison, 
            influence_on_test_loss_val,
            step_size=step_size)

    for num_to_poison in [0.1]:
        # If we're just attacking one training example, try attacking the first one and also the second one separately
        if num_to_poison == 0.1:
            indices_to_poison = all_indices_to_poison[0:1]
        elif num_to_poison == 1.2:
            indices_to_poison = all_indices_to_poison[1:2]
        else:
            indices_to_poison = all_indices_to_poison[:num_to_poison]
        orig_X_train_subset = np.copy(data_sets.train.x[indices_to_poison, :])
        orig_X_train_inception_features_subset = np.copy(inception_X_train.x[indices_to_poison, :])

        project_fn = get_projection_to_box_around_orig_point(orig_X_train_subset, box_radius_in_pixels=0.5)

        attack_success = iterative_attack(top_model, full_model, top_graph, full_graph, project_fn, 
                                          [test_idx], 
                                          test_description, 
                                          data_sets.train, data_sets.test, dataset_name,
                                          indices_to_poison=indices_to_poison,
                                          num_iter=100,
                                          step_size=step_size,
                                          save_iter=100,
                                          early_stop=0.5)

        with full_graph.as_default():
            data_sets.train.x[indices_to_poison, :] = orig_X_train_subset
            full_model.load_weights_from_disk(orig_weight_path, do_save=False, do_check=False)
        with top_graph.as_default():
            inception_X_train.x[indices_to_poison, :] = orig_X_train_inception_features_subset
            top_model.retrain_and_get_weights(inception_X_train.x, inception_X_train.labels)

        if attack_success:
            break

*** Full:


[genericNeuralNet.py:193 -   get_vec_to_list_fn() ] Total number of parameters: 2048


*** Top:


[genericNeuralNet.py:193 -   get_vec_to_list_fn() ] Total number of parameters: 2048


Using normal model


...

Norm of test gradient: 0.06544987
Loaded inverse HVP from output/dogfish_400_200_inception_wd-0.001-test-3.npz
Inverse HVP took 0.000939130783081 sec
Entering the for loop


[iter_attack.py:142 -     iterative_attack() ] Attach_iter 54 perturbation shape: (1, 268203), perturbation: [[-0.00579451  0.00146253  0.00298796 ... -0.00924681  0.00729955
   0.00270696]]
[iter_attack.py:67 - poison_with_influence_proj_gradient_step() ] -- max: 4.919024467468262, mean: 0.00010703031239584766, min: -4.619388103485107
[genericNeuralNet.py:330 -     print_model_eval() ] Train loss (w reg) on all data: 0.008364571
[genericNeuralNet.py:331 -     print_model_eval() ] Train loss (w/o reg) on all data: 0.002291307
[genericNeuralNet.py:333 -     print_model_eval() ] Test loss (w/o reg) on all data: 0.022337439
[genericNeuralNet.py:334 -     print_model_eval() ] Train acc on all data:  1.0
[genericNeuralNet.py:335 -     print_model_eval() ] Test acc on all data:   0.995
[genericNeuralNet.py:337 -     print_model_eval() ] Norm of the mean of gradients: 0.000605233
[genericNeuralNet.py:338 -     print_model_eval() ] Norm of the params: 3.485187


Using normal model
LBFGS training took [24] iter.
After training with LBFGS: 


[iter_attack.py:174 -     iterative_attack() ] Test pred (full): [[9.991646e-01 8.353787e-04]]
[iter_attack.py:177 -     iterative_attack() ] Test pred (top): [[9.9916995e-01 8.3006796e-04]]
[iter_attack.py:125 -     iterative_attack() ] *** Iter: 55


Total number of parameters: 2048
         Current function value: -1.994653
         Iterations: 7
         Function evaluations: 100
         Gradient evaluations: 95
         Hessian evaluations: 46
Inverse HVP took 0.628654003143 sec
